In [202]:
import torch
import tiktoken
import re

In [203]:
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [204]:
query = inputs[1]
att_scores = torch.empty(inputs.shape[0])

for idx,input in enumerate(inputs):
    att_scores[idx] = torch.dot(query, input)

att_scores

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])

In [205]:
att_wei = att_scores / att_scores.sum()
att_wei
# att_wei.sum()

tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])

In [206]:
def softmax_na(x):
    return torch.exp(x) / torch.sum(torch.exp(x), dim=0)


softmax_na(att_scores)
softmax_na(att_scores).sum()


tensor(1.)

In [207]:
torch.softmax(att_scores,dim=0)

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

In [208]:
att_wei = torch.softmax(att_scores,dim=0)

context_vec = torch.zeros(inputs[0].shape)

for idx,input in enumerate(inputs):
    context_vec = context_vec + att_wei[idx] * input

context_vec

tensor([0.4419, 0.6515, 0.5683])

In [209]:
inputs

scores = torch.empty((inputs.shape[0],inputs.shape[0]))

for idx,input in enumerate(inputs):
    scores[idx] = input @ inputs.T
    print(input @ inputs.T)

scores = inputs @ inputs.T
    
print(scores)


print(torch.softmax(torch.tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865]),dim=0))

print(torch.softmax(scores,dim=1))

print(torch.softmax(scores,dim=1)[1].sum())

tensor([0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310])
tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])
tensor([0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605])
tensor([0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565])
tensor([0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935])
tensor([0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450])
tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])
tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1

In [210]:
weight = torch.softmax(scores,dim=1)

weight.shape

context_vec = weight @ inputs
context_vec

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

In [211]:
torch.manual_seed(123)

d_in = inputs.shape[1]
d_out = 2

w_q = torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)
w_k = torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)
w_v = torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)

w_q,w_k,w_v

(Parameter containing:
 tensor([[0.2961, 0.5166],
         [0.2517, 0.6886],
         [0.0740, 0.8665]]),
 Parameter containing:
 tensor([[0.1366, 0.1025],
         [0.1841, 0.7264],
         [0.3153, 0.6871]]),
 Parameter containing:
 tensor([[0.0756, 0.1966],
         [0.3164, 0.4017],
         [0.1186, 0.8274]]))

In [212]:
x_2 = inputs[1]

q_2 = x_2 @ w_q
k_2 = x_2 @ w_k
v_2 = x_2 @ w_v

q_2,k_2,v_2

(tensor([0.4306, 1.4551]), tensor([0.4433, 1.1419]), tensor([0.3951, 1.0037]))

In [213]:
keys = inputs @ w_q
values = inputs @ w_v

keys,values

(tensor([[0.2309, 1.0966],
         [0.4306, 1.4551],
         [0.4300, 1.4343],
         [0.2355, 0.7990],
         [0.2983, 0.6565],
         [0.2568, 1.0533]]),
 tensor([[0.1855, 0.8812],
         [0.3951, 1.0037],
         [0.3879, 0.9831],
         [0.2393, 0.5493],
         [0.1492, 0.3346],
         [0.3221, 0.7863]]))

In [214]:
k_2 = keys[1]
ant_score_22 = q_2.dot(k_2)
print(ant_score_22)

ant_score_2 = q_2 @ keys.T
print(ant_score_2)



tensor(2.3026)
tensor([1.6951, 2.3026, 2.2722, 1.2640, 1.0838, 1.6432])


In [215]:
d_k = keys.shape[1]

ant_weight_2 = torch.softmax(ant_score_2/d_k**0.5,dim=0)
ant_weight_2

tensor([0.1565, 0.2404, 0.2353, 0.1154, 0.1016, 0.1508])

In [216]:
context_vec_2 = ant_weight_2 @ values
context_vec_2

tensor([0.3067, 0.8265])

In [217]:
print(w_q)

query = inputs @ w_q
print(query)


ant_score = query @ keys.T
print(ant_score)

ant_wei = torch.softmax(ant_score/d_k**0.5, dim=1)
print(ant_wei)


ant_ctx_vec = ant_wei @ values

print(ant_ctx_vec)

Parameter containing:
tensor([[0.2961, 0.5166],
        [0.2517, 0.6886],
        [0.0740, 0.8665]])
tensor([[0.2309, 1.0966],
        [0.4306, 1.4551],
        [0.4300, 1.4343],
        [0.2355, 0.7990],
        [0.2983, 0.6565],
        [0.2568, 1.0533]])
tensor([[1.2559, 1.6951, 1.6722, 0.9305, 0.7889, 1.2143],
        [1.6951, 2.3026, 2.2722, 1.2640, 1.0838, 1.6432],
        [1.6722, 2.2722, 2.2421, 1.2472, 1.0700, 1.6211],
        [0.9305, 1.2640, 1.2472, 0.6938, 0.5948, 0.9020],
        [0.7889, 1.0838, 1.0700, 0.5948, 0.5200, 0.7681],
        [1.2143, 1.6432, 1.6211, 0.9020, 0.7681, 1.1753]])
tensor([[0.1615, 0.2204, 0.2168, 0.1283, 0.1161, 0.1569],
        [0.1565, 0.2404, 0.2353, 0.1154, 0.1016, 0.1508],
        [0.1567, 0.2395, 0.2344, 0.1160, 0.1023, 0.1511],
        [0.1631, 0.2065, 0.2040, 0.1380, 0.1286, 0.1598],
        [0.1630, 0.2008, 0.1988, 0.1421, 0.1348, 0.1606],
        [0.1615, 0.2187, 0.2153, 0.1295, 0.1178, 0.1571]])
tensor([[0.2997, 0.8094],
        [0.3067, 0

In [218]:
class SimpleAttV1:
    def __init__(self,d_in,d_out) -> None:
        self.d_in = d_in
        self.d_out = d_out

        torch.manual_seed(123)
        
        self.q = torch.nn.Parameter(torch.randn(d_in,d_out),requires_grad=False)
        self.k = torch.nn.Parameter(torch.randn(d_in,d_out),requires_grad=False)
        self.v = torch.nn.Parameter(torch.randn(d_in,d_out),requires_grad=False)


    def forward(self, x):
        x_q = x @ self.q
        x_k = x @ self.k
        x_v = x @ self.v

        print(self.q)
        
        print(x_q)
        x_ant_score = x_q @ x_k.T
        # print(x_ant_score)
        x_ant_wei = torch.softmax(x_ant_score/self.d_out**0.5,dim=1)
        # print(x_ant_wei)
        x_ctx_vec = x_ant_wei @ x_v

        return x_ctx_vec

In [219]:
d_in = 3
d_out = 2

sa = SimpleAttV1(d_in,d_out)

x_ctx = sa.forward(inputs)
x_ctx

Parameter containing:
tensor([[-0.1115,  0.1204],
        [-0.3696, -0.2404],
        [-1.1969,  0.2093]])
tensor([[-1.1686,  0.2019],
        [-1.1729, -0.0048],
        [-1.1438, -0.0018],
        [-0.6339, -0.0439],
        [-0.2979,  0.0535],
        [-0.9596, -0.0712]])


tensor([[0.2845, 0.4071],
        [0.2854, 0.4081],
        [0.2854, 0.4075],
        [0.2864, 0.3974],
        [0.2863, 0.3910],
        [0.2860, 0.4039]])

In [220]:
class SimpleAttV2:
    def __init__(self,d_in,d_out,bias=False) -> None:
        self.d_in = d_in
        self.d_out = d_out

        torch.manual_seed(123)
        
        self.q = torch.nn.Linear(d_in,d_out,bias=bias)
        self.k = torch.nn.Linear(d_in,d_out,bias=bias)
        self.v = torch.nn.Linear(d_in,d_out,bias=bias)


    def forward(self, x):
        x_q = self.q(x)
        x_k = self.k(x)
        x_v = self.v(x)

        # print(self.q)
        
        # print(x_q)
        x_ant_score = x_q @ x_k.T
        # print(x_ant_score)
        x_ant_wei = torch.softmax(x_ant_score/self.d_out**0.5,dim=1)
        # print(x_ant_wei)
        x_ctx_vec = x_ant_wei @ x_v

        return x_ctx_vec

In [221]:
d_in = 3
d_out = 2

sa = SimpleAttV2(d_in,d_out)

x_ctx = sa.forward(inputs)
x_ctx

tensor([[-0.5337, -0.1051],
        [-0.5323, -0.1080],
        [-0.5323, -0.1079],
        [-0.5297, -0.1076],
        [-0.5311, -0.1066],
        [-0.5299, -0.1081]], grad_fn=<MmBackward0>)

In [222]:
a = torch.rand(6,6)
print(a)
am = torch.triu(torch.ones(6,6))
print(am)
torch.masked_fill(a,am.bool(),0)

am.sum(dim=1,keepdim=True)

drop = torch.nn.Dropout(0.5)
drop(torch.ones(6,6))

tensor([[0.3821, 0.6605, 0.8536, 0.5932, 0.6367, 0.9826],
        [0.2745, 0.6584, 0.2775, 0.8573, 0.8993, 0.0390],
        [0.9268, 0.7388, 0.7179, 0.7058, 0.9156, 0.4340],
        [0.0772, 0.3565, 0.1479, 0.5331, 0.4066, 0.2318],
        [0.4545, 0.9737, 0.4606, 0.5159, 0.4220, 0.5786],
        [0.9455, 0.8057, 0.6775, 0.6087, 0.6179, 0.6932]])
tensor([[1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.]])


tensor([[2., 0., 2., 0., 2., 2.],
        [2., 2., 0., 2., 0., 0.],
        [0., 0., 0., 0., 0., 2.],
        [0., 2., 2., 0., 2., 2.],
        [2., 2., 2., 0., 0., 2.],
        [2., 0., 2., 0., 0., 0.]])

In [223]:
class SimpleAttV3:
    def __init__(self,d_in,d_out,bias=False) -> None:
        self.d_in = d_in
        self.d_out = d_out

        torch.manual_seed(123)
        
        self.q = torch.nn.Linear(d_in,d_out,bias=bias)
        self.k = torch.nn.Linear(d_in,d_out,bias=bias)
        self.v = torch.nn.Linear(d_in,d_out,bias=bias)


    def forward(self, x):
        x_q = self.q(x)
        x_k = self.k(x)
        x_v = self.v(x)

        x_ant_score = x_q @ x_k.T

        x_ant_wei = torch.softmax(x_ant_score/self.d_out**0.5,dim=1)
        
        print(x_ant_wei)

        # mask = torch.tril(torch.ones(x_ant_wei.shape))
        # x_ant_wei = x_ant_wei * mask

        # print(x_ant_wei)

        # x_ant_wei = x_ant_wei / (x_ant_wei.sum(dim=1,keepdim=True))

        # print(x_ant_wei)

        mask = torch.triu(torch.ones(x_ant_wei.shape),diagonal=1)

        x_ant_wei = torch.masked_fill(x_ant_wei,mask.bool(),-torch.inf)

        print(x_ant_wei)

        x_ant_wei = torch.softmax(x_ant_wei/x_ant_wei.shape[1]**0.5,dim=1)

        print(x_ant_wei)

        dorp = torch.nn.Dropout(0.5)
        x_ant_wei = drop(x_ant_wei)

        print(x_ant_wei)
        
        x_ctx_vec = x_ant_wei @ x_v

        return x_ctx_vec

In [224]:
d_in = 3
d_out = 2

sa = SimpleAttV3(d_in,d_out)

x_ctx = sa.forward(inputs)
x_ctx

tensor([[0.1717, 0.1762, 0.1761, 0.1555, 0.1627, 0.1579],
        [0.1636, 0.1749, 0.1746, 0.1612, 0.1605, 0.1652],
        [0.1637, 0.1749, 0.1746, 0.1611, 0.1606, 0.1651],
        [0.1636, 0.1704, 0.1702, 0.1652, 0.1632, 0.1674],
        [0.1667, 0.1722, 0.1721, 0.1618, 0.1633, 0.1639],
        [0.1624, 0.1709, 0.1706, 0.1654, 0.1625, 0.1682]],
       grad_fn=<SoftmaxBackward0>)
tensor([[0.1717,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.1636, 0.1749,   -inf,   -inf,   -inf,   -inf],
        [0.1637, 0.1749, 0.1746,   -inf,   -inf,   -inf],
        [0.1636, 0.1704, 0.1702, 0.1652,   -inf,   -inf],
        [0.1667, 0.1722, 0.1721, 0.1618, 0.1633,   -inf],
        [0.1624, 0.1709, 0.1706, 0.1654, 0.1625, 0.1682]],
       grad_fn=<MaskedFillBackward0>)
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4988, 0.5012, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3323, 0.3339, 0.3338, 0.0000, 0.0000, 0.0000],
        [0.2496, 0.2503, 0.2503, 0.2498, 0.0000, 0.0000

tensor([[-0.9038,  0.4432],
        [-0.4509,  0.2211],
        [-0.4758, -0.1316],
        [-0.5824,  0.0120],
        [-0.6190, -0.0521],
        [-0.1405, -0.0501]], grad_fn=<MmBackward0>)

In [225]:
# inputs = torch.stack((inputs,inputs),dim=0)
inputs.shape
batch =  torch.stack((inputs,inputs),dim=0)
batch.shape

torch.Size([2, 6, 3])

In [226]:
from torch import nn

wei = nn.Linear(3,2,bias=False)
ant = torch.rand(2,6,3)

ant = wei(ant)
ant.shape

ant.transpose(1, 2).shape

ant @ ant.transpose(1, 2)

tensor([[[ 0.1089,  0.0141,  0.0603,  0.0695,  0.0959,  0.0246],
         [ 0.0141,  0.0571, -0.0237,  0.0546,  0.0024,  0.0745],
         [ 0.0603, -0.0237,  0.0514,  0.0125,  0.0588, -0.0271],
         [ 0.0695,  0.0546,  0.0125,  0.0819,  0.0529,  0.0744],
         [ 0.0959,  0.0024,  0.0588,  0.0529,  0.0863,  0.0086],
         [ 0.0246,  0.0745, -0.0271,  0.0744,  0.0086,  0.0975]],

        [[ 0.0103,  0.0287,  0.0344,  0.0268,  0.0202,  0.0062],
         [ 0.0287,  0.0818,  0.0919,  0.0632,  0.0617,  0.0351],
         [ 0.0344,  0.0919,  0.1210,  0.1091,  0.0573, -0.0108],
         [ 0.0268,  0.0632,  0.1091,  0.1309,  0.0218, -0.0810],
         [ 0.0202,  0.0617,  0.0573,  0.0218,  0.0547,  0.0604],
         [ 0.0062,  0.0351, -0.0108, -0.0810,  0.0604,  0.1572]]],
       grad_fn=<UnsafeViewBackward0>)

In [227]:


class CauseAtt(nn.Module):
    def __init__(self, context_len,in_dim,out_dim, bias=False) -> None:
        super().__init__()

        self.context_len = context_len
        self.in_dim = in_dim
        self.out_dim = out_dim

        self.qw = nn.Linear(in_dim,out_dim,bias=bias)
        self.kw = nn.Linear(in_dim,out_dim,bias=bias)
        self.vw = nn.Linear(in_dim,out_dim,bias=bias)

        self.register_buffer('mask',
                             torch.triu(torch.ones(context_len,context_len)))

    def forward(self,x):
        b,tok,dim = x.shape

        q = self.qw(x)
        k = self.kw(x)
        v = self.vw(x)

        ant_socre = q @ k.transpose(1,2)
        ant_socre = torch.masked_fill(ant_socre,self.mask.bool(),-torch.inf)

        ant_wei = torch.softmax(ant_score/self.context_len**0.5,dim=1)        

        drop = torch.nn.Dropout(0.5)
        ant_wei = drop(ant_wei)

        return ant_wei @ v
        


        

In [228]:
sa = CauseAtt(6,3,2)

x_ctx = sa(batch)
x_ctx

tensor([[[-0.3438, -0.5346],
         [-0.2723, -0.4053],
         [-0.4415, -0.6877],
         [-0.1288, -0.1892],
         [-0.3877, -0.5945],
         [-0.1617, -0.2719]],

        [[-0.3438, -0.5346],
         [-0.2723, -0.4053],
         [-0.4415, -0.6877],
         [-0.1288, -0.1892],
         [-0.3877, -0.5945],
         [-0.1617, -0.2719]]], grad_fn=<UnsafeViewBackward0>)

In [229]:
a = torch.ones(3,4)
b = torch.zeros(3,4)

print(torch.stack([a,b],dim=1).shape)
print(torch.stack([a,b],dim=1))


print(torch.concat([a,b],dim=1).shape)
print(torch.concat([a,b],dim=1))

torch.Size([3, 2, 4])
tensor([[[1., 1., 1., 1.],
         [0., 0., 0., 0.]],

        [[1., 1., 1., 1.],
         [0., 0., 0., 0.]],

        [[1., 1., 1., 1.],
         [0., 0., 0., 0.]]])
torch.Size([3, 8])
tensor([[1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.]])


In [230]:
class MultiHeadAttWrap():
    def __init__(self,n_heads, context_len,in_dim,out_dim, bias=False):
        self.heads = [CauseAtt(context_len,in_dim,out_dim, bias) for _ in range(n_heads)]

    def forward(self,x):
        return torch.cat([h(x) for h in self.heads],dim=-1)

In [231]:
sa = MultiHeadAttWrap(2,6,3,1)

x_ctx = sa.forward(batch)
print(x_ctx.shape)
print(x_ctx)

torch.Size([2, 6, 2])
tensor([[[ 0.0599, -0.2556],
         [ 0.0415, -0.0851],
         [ 0.0787, -0.2344],
         [-0.0018, -0.2202],
         [ 0.0830, -0.2525],
         [ 0.0410, -0.0354]],

        [[ 0.0599, -0.2556],
         [ 0.0415, -0.0851],
         [ 0.0787, -0.2344],
         [-0.0018, -0.2202],
         [ 0.0830, -0.2525],
         [ 0.0410, -0.0354]]], grad_fn=<CatBackward0>)


In [232]:
class MultiHeadAtt(nn.Module):
    def __init__(self,heads,cnt_len,in_dim,out_dim,blas=False,drp=0.5) -> None:
        super().__init__()
        assert out_dim%heads==0

        self.heads = heads
        self.head_dim = out_dim//heads

        self.qw = nn.Linear(in_dim,out_dim,bias=blas)
        self.kw = nn.Linear(in_dim,out_dim,bias=blas)
        self.vw = nn.Linear(in_dim,out_dim,bias=blas)
        
        self.drop = nn.Dropout(drp)

        self.register_buffer('mask',torch.triu(torch.ones(cnt_len,cnt_len),1))

        self.out_prj = nn.Linear(out_dim,out_dim)

    
    def forward(self,x):
        b,cnt_len,in_dim = x.shape

        q = self.qw(x)
        k = self.kw(x)
        v = self.vw(x)

        q_v = q.view(b,cnt_len,self.heads,self.head_dim).transpose(1,2)
        k_v = k.view(b,cnt_len,self.heads,self.head_dim).transpose(1,2)
        v_v = v.view(b,cnt_len,self.heads,self.head_dim).transpose(1,2)

        ant_score = q_v @ k_v.transpose(-2,-1)

        mask = self.mask.bool()[:cnt_len,:cnt_len]
        ant_score = torch.masked_fill(ant_score,mask,-torch.inf)

        ant_score = torch.softmax(ant_score/v.shape[-1]**0.5,dim=-1)

        ant_score = self.drop(ant_score)

        ctx_vec = ant_score @ v_v
        ctx_vec = ctx_vec.transpose(1,2)

        ctx_vec = ctx_vec.contiguous().view(b,cnt_len,v.shape[-1])
        
        return self.out_prj(ctx_vec)

In [233]:
sa = MultiHeadAtt(2,6,3,2)

x_ctx = sa.forward(batch)
print(x_ctx.shape)
print(x_ctx)

torch.Size([2, 6, 2])
tensor([[[ 0.2675, -0.3195],
         [ 0.6425, -0.2047],
         [ 0.8067, -0.2182],
         [ 0.7203, -0.2808],
         [ 0.8400, -0.2096],
         [ 0.7415, -0.2432]],

        [[ 0.7044, -0.2822],
         [ 0.9940, -0.1084],
         [ 1.1206, -0.0351],
         [ 0.7568, -0.1935],
         [ 0.5909, -0.2919],
         [ 0.7036, -0.2538]]], grad_fn=<ViewBackward0>)
